# IR assignment-1 Q2

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import nltk
import os
import string
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [82]:
import os
import string
from collections import defaultdict
import pickle

# Load preprocessed dataset
data_dir = "/content/drive/MyDrive/ass_data/CSE508_Winter2023_Dataset"
files = os.listdir(data_dir)
preprocessed_data = {}
for file in files:
    with open(os.path.join(data_dir, file), "r") as f:
        preprocessed_data[file] = f.read()

# Create unigram inverted index
unigram_index = defaultdict(set)
for file, text in preprocessed_data.items():
    for word in text.split():
        unigram_index[word].add(file)

# Save unigram inverted index to disk
with open("/content/drive/MyDrive/ass_data/unigram_index.pkl", "wb") as f:
    pickle.dump(unigram_index, f)

# Load unigram inverted index from disk
with open("/content/drive/MyDrive/ass_data/unigram_index.pkl", "rb") as f:
    loaded_unigram_index = pickle.load(f)

# Print the loaded unigram inverted index
print(loaded_unigram_index)


defaultdict(<class 'set'>, {'heat': {'cranfield0262', 'cranfield0098', 'cranfield1217', 'cranfield0586', 'cranfield0269', 'cranfield1281', 'cranfield0270', 'cranfield0030', 'cranfield1381', 'cranfield1241', 'cranfield0061', 'cranfield0645', 'cranfield1191', 'cranfield1222', 'cranfield0366', 'cranfield0651', 'cranfield1213', 'cranfield0565', 'cranfield1161', 'cranfield1282', 'cranfield0085', 'cranfield0982', 'cranfield0062', 'cranfield0352', 'cranfield0072', 'cranfield0328', 'cranfield0021', 'cranfield0572', 'cranfield1002', 'cranfield0396', 'cranfield0542', 'cranfield0546', 'cranfield0073', 'cranfield0399', 'cranfield0295', 'cranfield0283', 'cranfield0054', 'cranfield1258', 'cranfield0789', 'cranfield1215', 'cranfield1250', 'cranfield0240', 'cranfield0959', 'cranfield0159', 'cranfield0383', 'cranfield0263', 'cranfield0620', 'cranfield0486', 'cranfield0077', 'cranfield0668', 'cranfield0407', 'cranfield0485', 'cranfield1279', 'cranfield1159', 'cranfield0980', 'cranfield1264', 'cranfield0

In [86]:
import pickle
import re
import os
import pickle
from nltk.corpus import stopwords
from collections import defaultdict

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def save_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

In [84]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [89]:
from nltk.stem import WordNetLemmatizer
import re
import string
from nltk.corpus import stopwords


def intersect(list1, list2):
    result = list(set(list1) & set(list2))
    return result

def union(list1, list2):
    return list(set(list1) | set(list2))

def complement(list1, all_docs):
    return list(set(all_docs) - set(list1))

## count comparisons for AND operation
def comparisons_and(list1,list2,num_comparisons):
    x = y = 0
    list1.sort()
    list2.sort()
    while(x < len(list1) and y < len(list2)):
      if(int(list1[x][-4:]) == int(list2[y][-4:])):
        num_comparisons += 1
        x += 1
        y += 1
      elif(int(list1[x][-4:]) < int(list2[y][-4:])):
        num_comparisons += 1
        x += 1
      else:
        num_comparisons += 1
        y += 1
    return num_comparisons

## count comparisons for OR operation 
def comparisons_or(list1,list2,num_comparisons):
    x = y = 0
    list1.sort()
    list2.sort()
    while(x < len(list1) and y < len(list2)):
      if(int(list1[x][-4:]) == int(list2[y][-4:])):
        num_comparisons += 1
        x += 1
        y += 1
      elif(int(list1[x][-4:]) < int(list2[y][-4:])):
        num_comparisons += 1
        x += 1
      else:
        num_comparisons += 1
        y += 1
    return num_comparisons

def retrieve_documents(query, operations, unigram_index, preprocessed_data):
    
    # Preprocess the query tokens
    lem = WordNetLemmatizer()
    query = query.lower()
    query = re.sub('[^A-Z a-z ]+', ' ', query)
    query = query.split()
    tokens = []
    for j in query :
        tokens.append(j)
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    tokens = [token for token in tokens if token.strip()]
    tokens = [lem.lemmatize(token) for token in tokens]
    print(tokens)
    query_tokens = tokens

    # Get the list of documents for each query token
    document_sets = [list(unigram_index[token]) for token in query_tokens if token in unigram_index]

    # Perform the operations in order
    result = []
    num_comparisons = 0
    i = 0

    for operation in operations:

        if operation == "AND":
            if document_sets:
                if i == 0 :
                  num_comparisons = comparisons_and(document_sets[0],document_sets[1],num_comparisons)
                  result = intersect(document_sets[0],document_sets[1])
                  i = i + 2
                else :
                  num_comparisons = comparisons_and(result,document_sets[i],num_comparisons)
                  result = intersect(result,document_sets[i])
                  i = i + 1

        elif operation == "OR":
            if document_sets:
                if i == 0 :
                  num_comparisons = comparisons_or(document_sets[0],document_sets[1],num_comparisons)
                  result = union(document_sets[0],document_sets[1])
                  i = i + 2
                else :
                  num_comparisons = comparisons_or(result,document_sets[i],num_comparisons)
                  result = union(result,document_sets[i])
                  i = i + 1

        elif operation == "AND NOT":
            if document_sets:
                if i == 0 :
                  complemented_set = complement(document_sets[1],files)
                  num_comparisons = comparisons_and(document_sets[0],complemented_set,num_comparisons)
                  result = intersect(document_sets[0],complemented_set)
                  i = i + 2
                else :
                  complemented_set = complement(document_sets[i],files)
                  num_comparisons = comparisons_and(document_sets[i],complemented_set,num_comparisons)
                  result = intersect(result,complemented_set)
                  i = i + 1

        elif operation == "OR NOT":
            if document_sets:
                if i == 0 :
                  complemented_set = complement(document_sets[1],files)
                  num_comparisons = comparisons_or(document_sets[0],complemented_set,num_comparisons)
                  result = union(document_sets[0],complemented_set)
                  i = i + 2
                else :
                  complemented_set = complement(document_sets[i],files)
                  num_comparisons = comparisons_or(document_sets[i],complemented_set,num_comparisons)
                  result = union(result,complemented_set)
                  i = i + 1
                

    return result, num_comparisons


if __name__ == "__main__":
    n = int(input().strip())
    for i in range(n):
        query = input().strip()
        operations = input().strip().split(",")
        result, number_of_comparisons = retrieve_documents(query, operations, loaded_unigram_index, preprocessed_data)
        print("Query", i + 1)
        print("Number of documents retrieved for query", i + 1, ":", len(result))
        print("Names of the documents retrieved for query", i + 1, ":", result)
        print("Number of comparisons required for query", i + 1, ":", number_of_comparisons)
        print()


2
simple shear is a flow
AND,OR
['simple', 'shear', 'flow']
Query 1
Number of documents retrieved for query 1 : 673
Names of the documents retrieved for query 1 : ['cranfield0098', 'cranfield0124', 'cranfield0132', 'cranfield0269', 'cranfield0057', 'cranfield1325', 'cranfield0270', 'cranfield0299', 'cranfield0224', 'cranfield0665', 'cranfield0313', 'cranfield0569', 'cranfield1323', 'cranfield0028', 'cranfield0213', 'cranfield0514', 'cranfield1235', 'cranfield1288', 'cranfield0933', 'cranfield0228', 'cranfield1078', 'cranfield0380', 'cranfield0234', 'cranfield0610', 'cranfield1225', 'cranfield0691', 'cranfield1260', 'cranfield0683', 'cranfield0191', 'cranfield0420', 'cranfield0941', 'cranfield0205', 'cranfield1318', 'cranfield1250', 'cranfield0064', 'cranfield0383', 'cranfield0222', 'cranfield0668', 'cranfield0485', 'cranfield0377', 'cranfield0388', 'cranfield0792', 'cranfield0216', 'cranfield0507', 'cranfield0980', 'cranfield0772', 'cranfield0453', 'cranfield0496', 'cranfield0466', 'cr